In [2]:
import torch
print(torch.cuda.is_available())  # Should return True if a GPU is available
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should return the name of your GPU (e.g., GTX 1070)

True
1
NVIDIA GeForce GTX 1070


In [3]:
import os
import random
import shutil

# Define the base directories
base_dir = 'celeba_Orig'
images_dir = os.path.join(base_dir, 'images')
output_dir = os.path.join(base_dir, 'reduced/images')
# Create new directory structure
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
val_dir = os.path.join(output_dir, 'val')

# Create the directories if they do not exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

print("Directory structure created.")

# Get all image files
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpg')]

#Behavior of the Dataset and Loader ==> Preprocessing (preprocess Method): The preprocess method divides the dataset into two parts:
# The first 2000 entries go into self.test_dataset. The rest go into self.train_dataset.
# This division is fixed and deterministic because the seed for random.shuffle is set using random.seed(1234).
num_test_images = 2000 #int(len(image_files) * 0.10)
test_images = random.sample(image_files, num_test_images)

# Move the images to the test folder
for image in test_images:
    shutil.copy(os.path.join(images_dir, image), os.path.join(test_dir, image))

print(f"Moved {num_test_images} images to the test folder.")

Directory structure created.
Moved 2000 images to the test folder.


In [4]:
import os
import pandas as pd

# Define the path to the file
file_path = "celeba_orig/list_attr_celeba.txt"  # Replace with the actual file path

# Read the file starting from the second line, and add 'image_id' as the first column
column_names = ['image_id'] + [
    "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
    "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
    "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
    "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
    "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
    "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
    "Wearing_Necktie", "Young"
]

# Read the file, skipping the first line and adding column names
df = pd.read_csv(file_path, sep=r'\s+', header=None, skiprows=2, names=column_names)

# Define file paths
reduced_images_dir = "celeba_orig/reduced/images/test"
reduced_attr_file = "celeba_orig/reduced/list_attr_celeba.txt"

# Step 1: Get the list of image files in the reduced directory
reduced_images = set(os.listdir(reduced_images_dir))  # Get filenames in the directory

# Step 2: Filter the dataframe for rows where the image ID is in the reduced images
df_filtered = df[df['image_id'].isin(reduced_images)]

# Step 3: Open the file in write mode
with open(reduced_attr_file, 'w') as f:
    # Write the number of rows in df_filtered
    f.write(f"{len(df_filtered)}\n")
    
    # Write the column names excluding 'image_id' (join the rest with spaces)
    f.write(" ".join(df_filtered.columns[1:]) + "\n")
    
    # Write the data in df_filtered (row by row) to avoid extra new lines
    for index, row in df_filtered.iterrows():
        # Write each row to the file, with space-separated values
        f.write(" ".join(row.astype(str).values[0:]) + "\n")

In [5]:
#   Mode Selection in __getitem__: If the mode is 'test', it fetches items from self.test_dataset.
!python ./stargan_code/main.py --mode test --dataset CelebA --image_size 128 --c_dim 5 \
                 --selected_attrs Black_Hair Blond_Hair Brown_Hair Male Young \
                 --model_save_dir="C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita" \
                 --result_dir='stargan_celeba_128/results' \
                 --celeba_image_dir="C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita/celeba_Orig/reduced/images/test" \
                 --attr_path="C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita/celeba_Orig/reduced/list_attr_celeba.txt"

Namespace(c_dim=5, c2_dim=8, celeba_crop_size=178, rafd_crop_size=256, image_size=128, g_conv_dim=64, d_conv_dim=64, g_repeat_num=6, d_repeat_num=6, lambda_cls=1, lambda_rec=10, lambda_gp=10, dataset='CelebA', batch_size=16, num_iters=200000, num_iters_decay=100000, g_lr=0.0001, d_lr=0.0001, n_critic=5, beta1=0.5, beta2=0.999, resume_iters=None, selected_attrs=['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young'], test_iters=200000, num_workers=1, mode='test', use_tensorboard=True, celeba_image_dir='C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita/celeba_Orig/reduced/images/test', attr_path='C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita/celeba_Orig/reduced/list_attr_celeba.txt', rafd_image_dir='data/RaFD/train', log_dir='stargan/logs', model_save_dir='C:/Users/Gordts-De Laender/Desktop/Papa/OU/Capita', sample_dir='stargan/samples', result_dir="'stargan_celeba_128/results'", log_step=10, sample_step=1000, model_save_step=10000, lr_update_step=1000)
Finished preprocessing the 

C:\Users\Gordts-De Laender\Desktop\Papa\OU\Capita\stargan_code\solver.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.G.load_state_dict(torch.load(G_path, map_loc

In [9]:
# Post-process the output for single transformation
import os
import random
import shutil

#single transformation: Black to blond hair
    # Define source and destination directories
source_dir = r"'stargan_celeba_128/results'"  # Source directory
destination_dir = r"celeba_Orig/reduced/images/single transformation" 
os.makedirs(destination_dir, exist_ok=True)

    # Iterate over files in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith("Blond_Hair.jpg"):  # Check if file ends with 'Blond_Hair.jpg'
        src_path = os.path.join(source_dir, filename)  # Full path to the source file
        dest_path = os.path.join(destination_dir, filename)  # Full path to the destination
        shutil.copy(src_path, dest_path)  # Copy file
        print(f"Copied {filename} to {destination_dir}")

Copied 000052_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000088_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000100_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000106_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000241_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000333_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000383_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000496_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000502_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000508_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000536_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000576_Blond_Hair.jpg to celeba_Orig/reduced/images/single transformation
Copied 000594_Blond_Hair.jpg

In [ ]:
# Post-process the output for single transformation
import os
import random
import shutil

#single transformation: Black to blond hair
    # Define source and destination directories
source_dir = r"'stargan_celeba_128/results'"  # Source directory
destination_dir = r"celeba_Orig/reduced/images/single transformation" 
os.makedirs(destination_dir, exist_ok=True)

    # Iterate over files in the source directory
for filename in os.listdir(source_dir):
    if filename.endswith("Blond_Hair.jpg"):  # Check if file ends with 'Blond_Hair.jpg'
        src_path = os.path.join(source_dir, filename)  # Full path to the source file
        dest_path = os.path.join(destination_dir, filename)  # Full path to the destination
        shutil.copy(src_path, dest_path)  # Copy file
        print(f"Copied {filename} to {destination_dir}")

#multi-transformation: Black to blond hair to young
